In [1]:
#!pip install lmdb
import chg_utils as charge_util
from ocpmodels.datasets import LmdbDataset
from ocpmodels.datasets import data_list_collater
from ase.atoms import Atoms
import numpy as np

In [5]:
charge_util.build_charge_lmdb(
    inpath = '../shared-scratch/ethan/sample/train', 
    outpath = '../chg/100/train',
    use_tqdm=True
)

100%|██████████| 90/90 [14:00<00:00,  9.34s/it]


In [6]:
charge_util.build_charge_lmdb(
    inpath = '../shared-scratch/ethan/sample/val', 
    outpath = '../chg/100/val',
    use_tqdm=True
)

100%|██████████| 10/10 [01:09<00:00,  6.91s/it]


In [8]:
dataset = LmdbDataset({'src':'../chg/100/train'})

In [9]:
data0 = dataset[0]
data1 = dataset[1]


#del(data0.charge_edges)
#del(data1.charge_edges)

#print(data0, '\n')
#print(data1)

#print(data0.edge_index)
#print(data0.charge_edges.T)
#print(data0.natoms)
print(data0)
print(len(dataset))

Data(atomic_numbers=[78], cell=[1, 3, 3], cell_offsets=[4738, 3], charge_density=[[[-2.49442823e-08 -3.17695856e-08  8.79381037e-09 ... -2.47132895e-08
   -2.47541597e-08 -1.24344537e-08]
  [-2.14530806e-08 -4.87459178e-08  1.03777076e-08 ... -3.81896904e-08
   -1.65225635e-08  1.34106298e-08]
  [-6.32068736e-09 -5.14173052e-08  3.44158214e-09 ... -4.27789347e-08
   -1.89376429e-08  2.90163276e-08]
  ...
  [-1.77627979e-08  2.22854585e-08  1.44753706e-08 ... -4.74574238e-09
   -4.42839060e-08 -6.19464163e-08]
  [-1.43095616e-08  1.20455944e-08  3.57200098e-09 ... -7.47653036e-09
   -4.64054934e-08 -5.43139133e-08]
  [-1.87760212e-08 -8.06352136e-09  2.77789897e-09 ... -1.35472235e-08
   -3.86809341e-08 -3.75389840e-08]]

 [[ 1.64020198e-08 -4.42360372e-08 -2.38675677e-08 ... -5.20745946e-08
   -7.09722930e-08  4.50249100e-09]
  [ 2.96968214e-08 -6.26585705e-08 -3.46873505e-08 ... -6.23262926e-08
   -7.30896533e-08  2.92416369e-08]
  [ 3.31642855e-08 -6.58961877e-08 -3.26991272e-08 ... 

In [10]:
batch = data_list_collater([dataset[0], dataset[1], dataset[2]])

In [11]:
b2cg = charge_util.BatchToChargeGraphs()

batch = b2cg.construct_probe_graphs(batch)

In [12]:
print(batch.natoms)
print(batch.charge_density)

tensor([78, 68, 51])
[array([[[-2.49442823e-08, -3.17695856e-08,  8.79381037e-09, ...,
         -2.47132895e-08, -2.47541597e-08, -1.24344537e-08],
        [-2.14530806e-08, -4.87459178e-08,  1.03777076e-08, ...,
         -3.81896904e-08, -1.65225635e-08,  1.34106298e-08],
        [-6.32068736e-09, -5.14173052e-08,  3.44158214e-09, ...,
         -4.27789347e-08, -1.89376429e-08,  2.90163276e-08],
        ...,
        [-1.77627979e-08,  2.22854585e-08,  1.44753706e-08, ...,
         -4.74574238e-09, -4.42839060e-08, -6.19464163e-08],
        [-1.43095616e-08,  1.20455944e-08,  3.57200098e-09, ...,
         -7.47653036e-09, -4.64054934e-08, -5.43139133e-08],
        [-1.87760212e-08, -8.06352136e-09,  2.77789897e-09, ...,
         -1.35472235e-08, -3.86809341e-08, -3.75389840e-08]],

       [[ 1.64020198e-08, -4.42360372e-08, -2.38675677e-08, ...,
         -5.20745946e-08, -7.09722930e-08,  4.50249100e-09],
        [ 2.96968214e-08, -6.26585705e-08, -3.46873505e-08, ...,
         -6.2326

In [ ]:
import lmdb
import pickle
from ocpmodels.preprocessing import AtomsToGraphs
import os
#import ocpmodels.charge.chg_utils as charge_util
from ocpmodels.datasets import LmdbDataset
#from ocpmodels.datasets import data_list_collater
from ase.atoms import Atoms
import numpy as np

#a2g = charge_util.AtomsToChargeGraphs()
a2g = AtomsToGraphs(max_neigh=100,
                    radius=6,
                    r_forces=False,
                    r_distances=False,
                    r_fixed=False,
                    r_energy=False)

db = lmdb.open(
        os.path.join('../chg/base_case/', 'charge.lmdb'),
        map_size=1099511627776 * 2,
        subdir=False,
        meminit=False,
        map_async=True,
    )

molecule = Atoms('HO',
                positions = [[0, 0, 1], [0, 0, 2]],
                cell = [10, 10, 8],
                pbc=True)

#molecule.charge_grid = [1, 1, 2]

#print(molecule)

data_object = a2g.convert(molecule)
data_object.charge_density = np.array([[[1,2]]])

fid = 0

print(data_object)

txn = db.begin(write=True)
txn.put(f"{fid}".encode("ascii"), pickle.dumps(data_object,protocol=-1))
txn.commit()

txn = db.begin(write=True)
txn.put(f'length'.encode('ascii'), pickle.dumps(1, protocol=-1))
txn.commit()

db.sync()
db.close()

dataset = LmdbDataset({'src':'../chg/base_case'})

print(len(dataset))
print(dataset[0])

In [ ]:
dataset = LmdbDataset({'src':'../chg/base_case'})

In [ ]:
print(len(dataset))

In [ ]:
print(dataset[0])